# Sudoku

In this notebook we're going to use Imandra to reason about the [classic kind of puzzle](https://en.wikipedia.org/wiki/Sudoku) that you can find everywhere.

(*note*: this example was adapted from code from [Koen Claessen](https://github.com/koengit/) and [Dan Rosén](https://github.com/danr))

We're going to define what a sudoku puzzle is, and how to _check_ if a given sudoku is a solution.
From that we can get Imandra to find solutions for us, without actually writing a sudoku solver.

## Helpers

We're going to define a sudoku as a 9×9 grid.

### Numbers

However, for now, the _bounded model checker_ that comes along with Imandra doesn't handle numbers. We do not need much here besides length, so a unary notation (classic [Peano arithmetic](https://en.wikipedia.org/wiki/Peano_axioms)) will do.

In [1]:
type nat = Z | S of nat;;

(* readability matters *)
let rec int_of_nat = function Z -> 0i | S n -> Caml.Int.(1i + int_of_nat n) [@@program]
let pp_nat out n = Format.fprintf out "%d" (int_of_nat n) [@@program];;
#install_printer pp_nat;;  

type nat = Z | S of nat
val int_of_nat : nat -> Caml.Int.t = <fun>
val pp_nat : Format.formatter -> nat -> unit = <fun>


In [2]:
let rec length = function
  | [] -> Z
  | _ :: tl -> S (length tl)
  
let n3 = S (S (S Z));;
let n6 = S (S (S n3));;
let n9 = S (S (S n6));;

val length : 'a list -> nat = <fun>
val n3 : nat = 3
val n6 : nat = 6
val n9 : nat = 9


termination proof Termination proof call `length (List.tl _x)` from `length _x` original length _x sub length (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.064s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 1817 mk clause 3 datatype occurs check 21 mk bool var 50 arith assert upper 5 datatype splits 3 decisions 7 arith add rows 7 propagations 2 conflicts 7 arith fixed eqs 4 datatype accessor ax 5 arith conflicts 1 datatype constructor ax 8 num allocs 919255 final checks 6 added eqs 33 del clause 1 arith eq adapter 3 memory 5.800000 max memory 5.800000 Expand start[0.064s]
 not (_x = []) && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> List.tl _x = []
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((not (_x = []) && Ordinal.count _x >= 0) && Ordinal.count (List.tl _x) >= 0)
 || List.tl _x = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list`_1225|
 … expansions unroll expr (|count_`ty_0 list`_1225| (|get.::.1_1211| _x_1216)) expansions unroll expr (|count_`ty_0 list`_1225| _x_1216) expansions unsat (let ((a!1 (= (|count_`ty_0 list`_1225| _x_1216)
 (+ 1 (|count_`ty_0 list`_1225| (|get.…

### Rows, columns, blocks

Sudokus have some constraints that work on rows, and some that work on columns.
Using a `transpose` function we can always work on rows.

In [3]:
(** helper for {!transpose} *)
let rec transpose3 = function
  | [] -> []
  | [] :: tl -> transpose3 tl
  | (_::t) :: tl -> t :: transpose3 tl

let rec get_heads = function                                                                       
  | [] -> []
  | [] :: tl -> get_heads tl
  | (h :: _) :: tl -> h :: get_heads tl                                                            
;;     

(** We need a custom termination function here *)
let measure_transpose = function
| [] -> 0
| x :: _ -> List.length x
;;

(** Transpose rows and columns in a list of lists *)
let rec transpose l =
  match l with
  | [] -> []
  | [] :: _ -> []
  | (x1 :: xs) :: xss ->
    (x1 :: get_heads xss) :: transpose (xs :: transpose3 xss)
[@@measure Ordinal.of_int (measure_transpose l)]
;;

val transpose3 : 'a list list -> 'a list list = <fun>
val get_heads : 'a list list -> 'a list = <fun>
val measure_transpose : 'a list list -> Z.t = <fun>
val transpose : 'a list list -> 'a list list = <fun>


termination proof Termination proof call `transpose3 (List.tl _x)` from `transpose3 _x` original transpose3 _x sub transpose3 (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [_x <> [] && List.hd _x = [] && not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 8 arith assert lower 13 arith pivots 8 rlimit count 6164 mk clause 41 datatype occurs check 36 mk bool var 128 arith assert upper 14 datatype splits 11 decisions 29 arith add rows 17 propagations 80 conflicts 13 arith fixed eqs 7 datatype accessor ax 14 arith conflicts 2 datatype constructor ax 23 num allocs 8722316 final checks 10 added eqs 104 del clause 8 arith eq adapter 10 memory 6.830000 max memory 9.600000 Expand start[0.010s]
 (_x <> [] && List.hd _x = [])
 && not (_x = [])
 && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((((_x <> [] && List.hd _x = []) && not (_x = [])) && Ordinal.count _x >= 0)
 && Ordinal.count (List.tl _x) >= 0)
 || not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1255|
 … expansions unroll expr (|count_`ty_0 list list`_1255| (|get.::.1_1238| _x_1244)) expansions unroll expr (|count_`ty_0 list list`_1255| _x_1244) expansions unsat (let ((a!1 (= (ite (>= (|count_`ty_0 list`_1257| |[]_2|) 0)
 (|count_`ty_0 list`_1… call `transpose3 (List.tl _x)` from `transpose3 _x` original transpose3 _x sub transpose3 (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x <> [] && List.hd _x = []) && not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.011s details Expand smt_stats num checks 8 arith assert lower 11 arith pivots 9 rlimit count 3037 mk clause 33 datatype occurs check 31 mk bool var 94 arith assert upper 12 datatype splits 10 decisions 32 arith add rows 16 propagations 55 conflicts 12 arith fixed eqs 5 datatype accessor ax 7 arith conflicts 2 datatype constructor ax 14 num allocs 3054596 final checks 8 added eqs 62 del clause 8 arith eq adapter 7 memory 9.600000 max memory 9.600000 Expand start[0.011s]
 not (_x <> [] && List.hd _x = [])
 && not (_x = [])
 && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 (((not (_x <> [] && List.hd _x = []) && not (_x = []))
 && Ordinal.count _x >= 0)
 && Ordinal.count (List.tl _x) >= 0)
 || not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1255|
 … expansions unroll expr (|count_`ty_0 list list`_1255| (|get.::.1_1238| _x_1244)) expansions unroll expr (|count_`ty_0 list list`_1255| _x_1244) expansions unsat (let ((a!1 (ite (>= (|count_`ty_0 list`_1257| (|ge

termination proof Termination proof call `get_heads (List.tl _x)` from `get_heads _x` original get_heads _x sub get_heads (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [_x <> [] && List.hd _x = [] && not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.013s details Expand smt_stats num checks 8 arith assert lower 13 arith pivots 8 rlimit count 6156 mk clause 41 datatype occurs check 36 mk bool var 128 arith assert upper 14 datatype splits 11 decisions 29 arith add rows 17 propagations 80 conflicts 13 arith fixed eqs 7 datatype accessor ax 14 arith conflicts 2 datatype constructor ax 23 num allocs 20359368 final checks 10 added eqs 104 del clause 8 arith eq adapter 10 memory 10.210000 max memory 10.210000 Expand start[0.013s]
 (_x <> [] && List.hd _x = [])
 && not (_x = [])
 && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((((_x <> [] && List.hd _x = []) && not (_x = [])) && Ordinal.count _x >= 0)
 && Ordinal.count (List.tl _x) >= 0)
 || not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1286|
 … expansions unroll expr (|count_`ty_0 list list`_1286| (|get.::.1_1269| _x_1275)) expansions unroll expr (|count_`ty_0 list list`_1286| _x_1275) expansions unsat (let ((a!1 (= (ite (>= (|count_`ty_0 list`_1288| |[]_2|) 0)
 (|count_`ty_0 list`_1… call `get_heads (List.tl _x)` from `get_heads _x` original get_heads _x sub get_heads (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x <> [] && List.hd _x = []) && not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.020s details Expand smt_stats num checks 8 arith assert lower 11 arith pivots 9 rlimit count 3037 mk clause 33 datatype occurs check 31 mk bool var 94 arith assert upper 12 datatype splits 10 decisions 32 arith add rows 16 propagations 55 conflicts 12 arith fixed eqs 5 datatype accessor ax 7 arith conflicts 2 datatype constructor ax 14 num allocs 15702519 final checks 8 added eqs 62 del clause 8 arith eq adapter 7 memory 7.370000 max memory 9.600000 Expand start[0.020s]
 not (_x <> [] && List.hd _x = [])
 && not (_x = [])
 && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 (((not (_x <> [] && List.hd _x = []) && not (_x = []))
 && Ordinal.count _x >= 0)
 && Ordinal.count (List.tl _x) >= 0)
 || not
 (((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = []))
 && not
 (not ((List.tl _x) <> [] && List.hd (List.tl _x) = [])
 && not (List.tl _x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1286|
 … expansions unroll expr (|count_`ty_0 list list`_1286| (|get.::.1_1269| _x_1275)) expansions unroll expr (|count_`ty_0 list list`_1286| _x_1275) expansions unsat (let ((a!1 (ite (>= (|count_`ty_0 list`_1288| (|get.::

termination proof Termination proof call `transpose ((List.tl (List.hd l)) :: (transpose3 (List.tl l)))` from `transpose l` original transpose l sub transpose ((List.tl (List.hd l)) :: (transpose3 (List.tl l))) original ordinal Ordinal.Int (if measure_transpose l >= 0 then measure_transpose l else 0) sub ordinal Ordinal.Int
(if measure_transpose ((List.tl (List.hd l)) :: (transpose3 (List.tl l))) >=
 0
 then measure_transpose ((List.tl (List.hd l)) :: (transpose3 (List.tl l)))
 else 0) path [not (l <> [] && List.hd l = []) && not (l = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.008s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 2 rlimit count 2106 mk clause 5 datatype occurs check 46 mk bool var 68 arith assert upper 3 datatype splits 11 decisions 15 arith add rows 2 propagations 4 conflicts 8 arith fixed eqs 1 datatype accessor ax 8 arith conflicts 1 datatype constructor ax 16 num allocs 35414099 final checks 11 added eqs 48 del clause 5 arith eq adapter 2 memory 8.100000 max memory 10.210000 Expand start[0.008s]
 not (l <> [] && List.hd l = [])
 && not (l = [])
 && (if (if l = [] then 0 else List.length (List.hd l)) >= 0
 then if l = [] then 0 else List.length (List.hd l) else 0)
 >= 0
 && (if (if (List.tl (List.hd l)) :: (transpose3 …) = [] then 0
 else List.length …)
 >= 0
 then
 if (List.tl (List.hd l)) :: (transpose3 …) = [] then 0
 else List.length …
 else 0)
 >= 0
 ==> not
 (not (true && List.tl (List.hd l) = [])
 && not ((List.tl (List.hd l)) :: (transpose3 …) = []))
 || Ordinal.( << )
 (Ordinal.Int
 (if (if (List.tl (List.hd l)) :: (transpose3 …) = [] then 0
 else List.length …)
 >= 0
 then
 if (List.tl (List.hd l)) :: (transpose3 …) = [] then 0
 else List.length …
 else 0))
 (Ordinal.Int
 (if (if l = [] then 0 else List.length (List.hd l)) >= 0
 then if l = [] then 0 else List.length (List.hd l) else 0)) simplify into (not
 (((not (l <> [] && List.hd l = []) && not (l = []))
 && (if (if l = [] then 0 else List.length (List.hd l)) >= 0
 then if l = [] then 0 else List.length (List.hd l) else 0)
 >= 0)
 && (if List.length (List.tl (List.hd l)) >= 0
 then List.length (List.tl (List.hd l)) else 0)
 >= 0)
 || List.tl (List.hd l) = [])
|| Ordinal.( << )
 (Ordinal.Int
 (if List.length (List.tl (List.hd l)) >= 0
 then List.length (List.tl (List.hd l)) else 0))
 (Ordinal.Int
 (if (if l = [] then 0 else List.length (List.hd l)) >= 0
 then if l = [] then 0 else List.length (List.hd l) else 0)) expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (>= (|List.length_1355| (|get.::.1_1329| (|get.::.0_1331| l_1348)))
 0))
 … expansions unroll expr (|List.length_1355| (|get.::.1_1329| (|get.::.0_1331| l_1348))) expansions unroll expr (|List.length_1355| (|get.::.0_1331| l_1348)) expansions unsat (let ((a!1 (+ 1 (|List.length_1355| (|get.::.1_1329| (|get.::.0_1331| l_1348)))))
 (a!7 (not (a…

Now we also need to extract 3×3 blocks for the additional constraint that none of them contains a duplicate.

This require a few helpers on lists and options, nothing too complicated.

In [4]:
let rec take (x:nat) l : _ list =
  match x with
  | Z -> []
  | S x' ->
    match l with
    | [] -> []
    | y :: tl -> y :: take x' tl

let rec drop x y =
  match x with
  | Z -> y
  | S x' ->
    match y with
    | [] -> []
    | _ :: y' -> drop x' y'

let rec elem x y =  match y with [] -> false | z :: ys -> x=z || elem x ys ;;

(** Is the list [l] composed of unique elements (without duplicates)? *)
let rec unique x : bool =
  match x with
  | [] -> true
  | y :: xs -> not (elem y xs) && unique xs
;;

(** Keep the elements that are [Some _], drop the others *)
let rec keep_some_list l =
  match l with
  | [] -> []
  | y :: tail ->
    let tail = keep_some_list tail in
    match y with None -> tail | Some x -> x :: tail
;;

(** A block is valid if it doesn't contain duplicates *)
let block_satisfies_constraints x = unique (keep_some_list x) ;;

let rec blocks_3_34 = function
  | [] -> []
  | y :: z -> drop n6 y :: blocks_3_34 z
;;

let rec blocks_3_33 = function
  | [] -> []
  | y :: z -> take n3 (drop n3 y) :: blocks_3_33 z
;;

let rec blocks_3_32 = function
  | [] -> []
  | y :: z -> take n3 y :: blocks_3_32 z
;;

(*

let rec group3 = function
  | xs1 :: xs2 :: xs3 :: xss ->
    (xs1 @ xs2 @ xs3) :: (group3 xss)
  | _ -> []
  ;;
*)

let rec group3 = function
  | [] -> []
  | xs1 :: y ->
    match y with
    | [] -> []
    | xs2 :: z ->
      match z with
      | [] -> []
      | xs3 :: xss -> (xs1 @ xs2 @ xs3) :: (group3 xss)
;;

let blocks_3_3 l =
  group3 (blocks_3_32 l) @
    group3 (blocks_3_33 l) @
      group3 (blocks_3_34 l)
;;

val take : nat -> 'a list -> 'a list = <fun>
val drop : nat -> 'a list -> 'a list = <fun>
val elem : 'a -> 'a list -> bool = <fun>
val unique : 'a list -> bool = <fun>
val keep_some_list : 'a option list -> 'a list = <fun>
val block_satisfies_constraints : 'a option list -> bool = <fun>
val blocks_3_34 : 'a list list -> 'a list list = <fun>
val blocks_3_33 : 'a list list -> 'a list list = <fun>
val blocks_3_32 : 'a list list -> 'a list list = <fun>
val group3 : 'a list list -> 'a list list = <fun>
val blocks_3_3 : 'a list list -> 'a list list = <fun>


termination proof Termination proof call `take (Destruct(S, 0, x)) (List.tl l)` from `take x l` original take x l sub take (Destruct(S, 0, x)) (List.tl l) original ordinal Ordinal.Int (Ordinal.count x) sub ordinal Ordinal.Int (Ordinal.count (Destruct(S, 0, x))) path [not (l = []) && not (x = Z)] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.026s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 2043 mk clause 3 datatype occurs check 30 mk bool var 66 arith assert upper 5 datatype splits 6 decisions 14 arith add rows 7 propagations 1 conflicts 10 arith fixed eqs 4 datatype accessor ax 8 arith conflicts 1 datatype constructor ax 16 num allocs 48868947 final checks 6 added eqs 52 del clause 1 arith eq adapter 3 memory 8.670000 max memory 10.210000 Expand start[0.026s]
 not (l = [])
 && not (x = Z)
 && Ordinal.count x >= 0 && Ordinal.count (Destruct(S, 0, x)) >= 0
 ==> not (not (List.tl l = []) && not (Destruct(S, 0, x) = Z))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (Destruct(S, 0, x))))
 (Ordinal.Int (Ordinal.count x)) simplify into (not
 (((not (l = []) && not (x = Z)) && Ordinal.count x >= 0)
 && Ordinal.count (Destruct(S, 0, x)) >= 0)
 || not (not (List.tl l = []) && not (Destruct(S, 0, x) = Z)))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (Destruct(S, 0, x))))
 (Ordinal.Int (Ordinal.count x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (count_nat_1387 (|get.S.0_1366| x_1376)))
 (|Or… expansions unroll expr (count_nat_1387 (|get.S.0_1366| x_1376)) expansions unroll expr (count_nat_1387 x_1376) expansions unsat (let ((a!1 (= (count_nat_1387 x_1376)
 (+ 1 (count_nat_1387 (|get.S.0_1366| x_1376)))))…

termination proof Termination proof call `drop (Destruct(S, 0, x)) (List.tl y)` from `drop x y` original drop x y sub drop (Destruct(S, 0, x)) (List.tl y) original ordinal Ordinal.Int (Ordinal.count x) sub ordinal Ordinal.Int (Ordinal.count (Destruct(S, 0, x))) path [not (y = []) && not (x = Z)] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.018s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 2043 mk clause 3 datatype occurs check 30 mk bool var 66 arith assert upper 5 datatype splits 6 decisions 14 arith add rows 7 propagations 1 conflicts 10 arith fixed eqs 4 datatype accessor ax 8 arith conflicts 1 datatype constructor ax 16 num allocs 64138127 final checks 6 added eqs 52 del clause 1 arith eq adapter 3 memory 9.080000 max memory 10.210000 Expand start[0.018s]
 not (y = [])
 && not (x = Z)
 && Ordinal.count x >= 0 && Ordinal.count (Destruct(S, 0, x)) >= 0
 ==> not (not (List.tl y = []) && not (Destruct(S, 0, x) = Z))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (Destruct(S, 0, x))))
 (Ordinal.Int (Ordinal.count x)) simplify into (not
 (((not (y = []) && not (x = Z)) && Ordinal.count x >= 0)
 && Ordinal.count (Destruct(S, 0, x)) >= 0)
 || not (not (List.tl y = []) && not (Destruct(S, 0, x) = Z)))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (Destruct(S, 0, x))))
 (Ordinal.Int (Ordinal.count x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (count_nat_1413 (|get.S.0_1393| x_1402)))
 (|Or… expansions unroll expr (count_nat_1413 (|get.S.0_1393| x_1402)) expansions unroll expr (count_nat_1413 x_1402) expansions unsat (let ((a!1 (= (count_nat_1413 x_1402)
 (+ 1 (count_nat_1413 (|get.S.0_1393| x_1402)))))…

termination proof Termination proof call `elem x (List.tl y)` from `elem x y` original elem x y sub elem x (List.tl y) original ordinal Ordinal.Int (Ordinal.count y) sub ordinal Ordinal.Int (Ordinal.count (List.tl y)) path [not (x = List.hd y) && not (y = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.022s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 1936 mk clause 3 datatype occurs check 18 mk bool var 52 arith assert upper 5 datatype splits 3 decisions 7 arith add rows 7 propagations 2 conflicts 7 arith fixed eqs 4 datatype accessor ax 5 arith conflicts 1 datatype constructor ax 8 num allocs 81557087 final checks 5 added eqs 33 del clause 1 arith eq adapter 3 memory 9.430000 max memory 12.200000 Expand start[0.022s]
 not (x = List.hd y)
 && not (y = []) && Ordinal.count y >= 0 && Ordinal.count (List.tl y) >= 0
 ==> not (not (x = List.hd (List.tl y)) && not (List.tl y = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl y)))
 (Ordinal.Int (Ordinal.count y)) simplify into (not
 (((not (x = List.hd y) && not (y = [])) && Ordinal.count y >= 0)
 && Ordinal.count (List.tl y) >= 0)
 || not (not (x = List.hd (List.tl y)) && not (List.tl y = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl y)))
 (Ordinal.Int (Ordinal.count y)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list`_1438|
 … expansions unroll expr (|count_`ty_0 list`_1438| (|get.::.1_1421| y_1428)) expansions unroll expr (|count_`ty_0 list`_1438| y_1428) expansions unsat (let ((a!1 (= (|count_`ty_0 list`_1438| y_1428)
 (+ 1 (|count_`ty_0 list`_1438| (|get.:…

termination proof Termination proof call `unique (List.tl x)` from `unique x` original unique x sub unique (List.tl x) original ordinal Ordinal.Int (Ordinal.count x) sub ordinal Ordinal.Int (Ordinal.count (List.tl x)) path [not (elem (List.hd x) (List.tl x)) && not (x = [])] proof detailed proof summary full ground_instances 4 definitions 0 inductions 0 search_time 0.009s details Expand smt_stats num checks 9 arith assert lower 6 arith pivots 4 rlimit count 2357 mk clause 8 datatype occurs check 21 mk bool var 58 arith assert upper 4 datatype splits 3 decisions 15 arith add rows 5 propagations 5 conflicts 7 arith fixed eqs 3 datatype accessor ax 5 arith conflicts 1 datatype constructor ax 6 num allocs 92848036 final checks 6 added eqs 31 del clause 1 arith eq adapter 3 memory 12.690000 max memory 12.690000 Expand start[0.009s]
 not (elem (List.hd x) (List.tl x))
 && not (x = []) && Ordinal.count x >= 0 && Ordinal.count (List.tl x) >= 0
 ==> not
 (not (elem (List.hd (List.tl x)) (List.tl (List.tl x)))
 && not (List.tl x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl x)))
 (Ordinal.Int (Ordinal.count x)) simplify into (not
 (((not (elem (List.hd x) (List.tl x)) && not (x = []))
 && Ordinal.count x >= 0)
 && Ordinal.count (List.tl x) >= 0)
 || not
 (not (elem (List.hd (List.tl x)) (List.tl (List.tl x)))
 && not (List.tl x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl x)))
 (Ordinal.Int (Ordinal.count x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list`_1464|
 … expansions unroll expr (elem_1450 (|get.::.0_1444| (|get.::.1_1445| x_1455))
 (|get.::.1_1445| (|get.::.1_1445| x… expansions unroll expr (|count_`ty_0 list`_1464| (|get.::.1_1445| x_1455)) expansions unroll expr (|count_`ty_0 list`_1464| x_1455) expansions unsat (let ((a!1 (= (|count_`ty_0 list`_1464| x_1455)
 (+ 1 (|count_`ty_0 list`_1464| (|get.:…

termination proof Termination proof call `keep_some_list (List.tl l)` from `keep_some_list l` original keep_some_list l sub keep_some_list (List.tl l) original ordinal Ordinal.Int (Ordinal.count l) sub ordinal Ordinal.Int (Ordinal.count (List.tl l)) path [not (l = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 1978 mk clause 3 datatype occurs check 29 mk bool var 64 arith assert upper 5 datatype splits 9 decisions 16 arith add rows 7 propagations 2 conflicts 7 arith fixed eqs 4 datatype accessor ax 10 arith conflicts 1 datatype constructor ax 16 num allocs 113173087 final checks 8 added eqs 49 del clause 1 arith eq adapter 3 memory 13.040000 max memory 13.040000 Expand start[0.010s]
 not (l = []) && Ordinal.count l >= 0 && Ordinal.count (List.tl l) >= 0
 ==> List.tl l = []
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl l)))
 (Ordinal.Int (Ordinal.count l)) simplify into (not
 ((not (l = []) && Ordinal.count l >= 0) && Ordinal.count (List.tl l) >= 0)
 || List.tl l = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl l)))
 (Ordinal.Int (Ordinal.count l)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 option list`_1493|
 … expansions unroll expr (|count_`ty_0 option list`_1493| (|get.::.1_1474| l_1484)) expansions unroll expr (|count_`ty_0 option list`_1493| l_1484) expansions unsat (let ((a!1 (= (|count_`ty_0 option list`_1493| l_1484)
 (+ 2 (|count_`ty_0 option list`…

termination proof Termination proof call `blocks_3_34 (List.tl _x)` from `blocks_3_34 _x` original blocks_3_34 _x sub blocks_3_34 (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.011s details Expand smt_stats num checks 8 arith assert lower 15 arith pivots 8 rlimit count 2597 mk clause 12 datatype occurs check 29 mk bool var 87 arith assert upper 12 datatype splits 9 decisions 24 arith add rows 12 propagations 10 conflicts 10 arith fixed eqs 7 datatype accessor ax 10 arith conflicts 2 arith assert diseq 1 datatype constructor ax 16 num allocs 135470915 final checks 8 added eqs 58 del clause 5 arith eq adapter 10 memory 13.710000 max memory 13.710000 Expand start[0.011s]
 not (_x = []) && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> List.tl _x = []
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((not (_x = []) && Ordinal.count _x >= 0) && Ordinal.count (List.tl _x) >= 0)
 || List.tl _x = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1552|
 … expansions unroll expr (|count_`ty_0 list list`_1552| (|get.::.1_1532| _x_1543)) expansions unroll expr (|count_`ty_0 list list`_1552| _x_1543) expansions unsat (let ((a!1 (ite (>= (|count_`ty_0 list`_1554| (|get.::.0_1531| _x_1543)) 0)
 (|count_…

termination proof Termination proof call `blocks_3_33 (List.tl _x)` from `blocks_3_33 _x` original blocks_3_33 _x sub blocks_3_33 (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 8 arith assert lower 15 arith pivots 8 rlimit count 2641 mk clause 12 datatype occurs check 29 mk bool var 87 arith assert upper 12 datatype splits 9 decisions 24 arith add rows 15 propagations 10 conflicts 10 arith fixed eqs 7 datatype accessor ax 10 arith conflicts 2 arith assert diseq 1 datatype constructor ax 16 num allocs 150083912 final checks 8 added eqs 58 del clause 5 arith eq adapter 10 memory 17.040000 max memory 17.040000 Expand start[0.010s]
 not (_x = []) && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> List.tl _x = []
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((not (_x = []) && Ordinal.count _x >= 0) && Ordinal.count (List.tl _x) >= 0)
 || List.tl _x = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1590|
 … expansions unroll expr (|count_`ty_0 list list`_1590| (|get.::.1_1564| _x_1581)) expansions unroll expr (|count_`ty_0 list list`_1590| _x_1581) expansions unsat (let ((a!1 (ite (>= (|count_`ty_0 list`_1592| (|get.::.0_1563| _x_1581)) 0)
 (|count_…

termination proof Termination proof call `blocks_3_32 (List.tl _x)` from `blocks_3_32 _x` original blocks_3_32 _x sub blocks_3_32 (List.tl _x) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl _x)) path [not (_x = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.009s details Expand smt_stats num checks 8 arith assert lower 15 arith pivots 8 rlimit count 2597 mk clause 12 datatype occurs check 29 mk bool var 87 arith assert upper 12 datatype splits 9 decisions 24 arith add rows 12 propagations 10 conflicts 10 arith fixed eqs 7 datatype accessor ax 10 arith conflicts 2 arith assert diseq 1 datatype constructor ax 16 num allocs 187042951 final checks 8 added eqs 58 del clause 5 arith eq adapter 10 memory 14.770000 max memory 17.040000 Expand start[0.009s]
 not (_x = []) && Ordinal.count _x >= 0 && Ordinal.count (List.tl _x) >= 0
 ==> List.tl _x = []
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((not (_x = []) && Ordinal.count _x >= 0) && Ordinal.count (List.tl _x) >= 0)
 || List.tl _x = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl _x)))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list list`_1623|
 … expansions unroll expr (|count_`ty_0 list list`_1623| (|get.::.1_1602| _x_1614)) expansions unroll expr (|count_`ty_0 list list`_1623| _x_1614) expansions unsat (let ((a!1 (ite (>= (|count_`ty_0 list`_1625| (|get.::.0_1601| _x_1614)) 0)
 (|count_…

termination proof Termination proof call `group3 (List.tl (List.tl (List.tl _x)))` from `group3 _x` original group3 _x sub group3 (List.tl (List.tl (List.tl _x))) original ordinal Ordinal.Int (Ordinal.count _x) sub ordinal Ordinal.Int (Ordinal.count (List.tl (List.tl (List.tl _x)))) path [not (List.tl (List.tl _x) = []) && not (List.tl _x = []) && not (_x = [])] proof detailed proof summary full ground_instances 12 definitions 0 inductions 0 search_time 0.033s details Expand smt_stats arith offset eqs 98 num checks 26 arith assert lower 487 arith pivots 125 rlimit count 45098 mk clause 546 datatype occurs check 210 mk bool var 1201 arith assert upper 467 datatype splits 109 decisions 1043 arith add rows 600 arith bound prop 67 propagations 1328 interface eqs 19 conflicts 101 arith fixed eqs 207 datatype accessor ax 121 minimized lits 26 arith conflicts 32 arith assert diseq 164 datatype constructor ax 312 num allocs 218446308 final checks 56 added eqs 1663 del clause 277 arith eq adapter 317 memory 16.020000 max memory 17.040000 Expand start[0.033s]
 not (List.tl (List.tl _x) = [])
 && not (List.tl _x = [])
 && not (_x = [])
 && Ordinal.count _x >= 0
 && Ordinal.count (List.tl (List.tl (List.tl _x))) >= 0
 ==> not
 (not (List.tl (List.tl (List.tl (List.tl (List.tl _x)))) = [])
 && not (List.tl (List.tl (List.tl (List.tl _x))) = [])
 && not (List.tl (List.tl (List.tl _x)) = []))
 || Ordinal.( << )
 (Ordinal.Int (Ordinal.count (List.tl (List.tl (List.tl _x)))))
 (Ordinal.Int (Ordinal.count _x)) simplify into (not
 ((((not (List.tl (List.tl _x) = []) && not (List.tl _x = []))
 && not (_x = []))
 && Ordinal.count _x >= 0)
 && Ordinal.count (List.tl (List.tl (List.tl _x))) >= 0)
 || not
 ((not (List.tl (List.tl (List.tl (List.tl (List.tl _x)))) = [])
 && not (List.tl (List.tl (List.tl (List.tl _x))) = []))
 && not (List.tl (List.tl (List.tl _x)) = [])))
|| Ordinal.( << )
 (Ordinal.Int (Ordinal.count (List.tl (List.tl (List.tl _x)))))
 (Ordinal.Int (Ordinal.count _x)) expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (|count_`ty_0 list list`_1658|
 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_… expansions unroll expr (|count_`ty_0 list list`_1658|
 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| _x_1649)))) expansions unroll expr (|count_`ty_0 list list`_1658| _x_1649) expansions unroll expr (let ((a!1 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| _x_1649))))))
 (|cou… expansions unroll expr (let ((a!1 (|count_`ty_0 list list`_1658|
 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_… expansions unroll expr (let ((a!1 (|get.::.0_1634| (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| _x_1649))))))
 (|cou… expansions unroll expr (|count_`ty_0 list list`_1658| (|get.::.1_1635| _x_1649)) expansions unroll expr (|count_`ty_0 list`_1660| (|get.::.0_1634| _x_1649)) expansions unroll expr (let ((a!1 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| _x_1649))))))
 (|cou… expansions unroll expr (let ((a!1 (|count_`ty_0 list list`_1658|
 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_… expansions unroll expr (let ((a!1 (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| (|get.::.1_1635| _x_1649))))))
 (|cou… expansions unroll expr (|count_`ty_0 list list`_1658| (|get.::.1_1635| (|get.::.1_1635| _x_1649))) expansions unsat (let ((a!1 (>= (|count_`ty_0 list`_1660|
 (|get.::.0_1634| (|get.::.1_1635| _x_1649)…

## The Sudoku type

We're ready to define the sudoku as a list of lists of (possibly empty) cells.

First, cells are just an enumeration of 9 distinct cases:

In [5]:
type cell = C1 | C2 | C3 | C4 | C5 | C6 | C7 | C8 | C9 ;;

(* let us also write a nice printer for cells. We will put
   it to good use later. *)
let doc_of_cell c =
  Document.s (match c with C1->"1"|C2->"2"|C3->"3"|C4->"4"|C5->"5"|C6->"6"|C7->"7"|C8->"8"|C9->"9") [@@program];;
  
#install_doc doc_of_cell;;

type cell = C1 | C2 | C3 | C4 | C5 | C6 | C7 | C8 | C9
val doc_of_cell : cell -> Imandra_surface.Document.t = <fun>


And the sudoku itself:

In [6]:
type sudoku = { rows: cell option list list } ;;

(* now install a nice printer *)

let doc_of_sudoku (s:sudoku) : Document.t =
  let module D = Document in
  let d_of_c = function None -> D.s "·" | Some c -> doc_of_cell c in 
  D.tbl_of d_of_c s.rows [@@program]
  ;;
  
#install_doc doc_of_sudoku;;

type sudoku = { rows : cell option list list; }
val doc_of_sudoku : sudoku -> Imandra_surface.Document.t = <fun>


We're going to solve the following instance (still from Dan Rosén and Koen Claessen's code).

In [7]:
let the_problem : sudoku =
  {rows=
    [ [ (Some C8) ; None ; None ; None ; None ; None ; None ; None ; None ];
    [ None ; None ; (Some C3) ; (Some C6) ; None ; None ; None ; None ; None ];
    [ None ; (Some C7) ; None ; None ; (Some C9) ; None ; (Some C2) ; None ; None ];
    [ None ; (Some C5) ; None ; None ; None ; (Some C7) ; None ; None ; None ];
    [ None ; None ; None ; None ; (Some C4) ; (Some C5) ; (Some C7) ; None ; None ];
    [ None ; None ; None ; (Some C1) ; None ; None ; None ; (Some C3) ; None ];
    [ None ; None ; (Some C1) ; None ; None ; None ; None ; (Some C6) ; (Some C8); ];
    [ None ; None ; (Some C8) ; (Some C5) ; None ; None ; None ; (Some C1) ; None ];
    [ None ; (Some C9) ; None ; None ; None ; None ; (Some C4) ; None ; None ];
  ]}
;;

val the_problem : sudoku = <document>


8,·,·,·,·,·,·,·,·
·,·,3,6,·,·,·,·,·
·,7,·,·,9,·,2,·,·
·,5,·,·,·,7,·,·,·
·,·,·,·,4,5,7,·,·
·,·,·,1,·,·,·,3,·
·,·,1,·,·,·,·,6,8
·,·,8,5,·,·,·,1,·
·,9,·,·,·,·,4,·,·


In [8]:
(** All the relevant blocks: rows, columns, and 3×3 sub-squares *)
let blocks (x:sudoku) =
  x.rows @ transpose x.rows @ blocks_3_3 x.rows

(** Are all constraints satisfied? *)
let satisfies_constraints (x:sudoku) = List.for_all block_satisfies_constraints (blocks x);;

(** is a sudoku entirely defined (all cells are filled)? *)
let is_solved (x:sudoku) =
  List.for_all (List.for_all Option.is_some) x.rows;;
  
(** Is [x] of the correct shape, i.e. a 9×9 grid? *)
let is_valid_sudoku (x:sudoku) =
  length x.rows = n9 &&
  List.for_all (fun col -> length col = n9) x.rows
;;

val blocks : sudoku -> cell option list list = <fun>
val satisfies_constraints : sudoku -> bool = <fun>
val is_solved : sudoku -> bool = <fun>
val is_valid_sudoku : sudoku -> bool = <fun>


We have a template (the initial problem) and we want to solve it.
It means the sudoku we're looking for must be:

- solved (all cells are `Some _` rather than `None`)
- a solution of the template (i.e. cells defined in the template must match)

In [9]:
(** Combine lists together *)
let rec zip l1 l2 = match l1, l2 with
  | [], _ | _, [] -> []
  | x1::tl1, x2 :: tl2 -> (x1,x2) :: zip tl1 tl2

let rec match_cols y =
  match y with
  | [] -> true
  | z :: x2 ->
    match z with
    | None,_ | _, None -> match_cols x2
    | (Some n1,Some n2) -> n1=n2 && match_cols x2
;;

let rec match_rows x =
  match x with
  | [] -> true
  | (row1,row2) :: z -> match_cols (zip row1 row2) && match_rows z
;;

(** is [x] a solution of [y]? We check that each cell in each rows,
    if defined in [y], has the same value in [x] *)
let is_solution_of (x:sudoku) (y:sudoku) : bool =
  is_solved x &&
  satisfies_constraints x &&
  match_rows (zip x.rows y.rows)


val zip : 'a list -> 'b list -> ('a * 'b) list = <fun>
val match_cols : ('a option * 'a option) list -> bool = <fun>
val match_rows : ('a option list * 'a option list) list -> bool = <fun>
val is_solution_of : sudoku -> sudoku -> bool = <fun>


termination proof Termination proof call `zip (List.tl l1) (List.tl l2)` from `zip l1 l2` original zip l1 l2 sub zip (List.tl l1) (List.tl l2) original ordinal Ordinal.Int (Ordinal.count l1) sub ordinal Ordinal.Int (Ordinal.count (List.tl l1)) path [not (l1 = [] || l2 = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.024s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 3 rlimit count 2037 mk clause 3 datatype occurs check 30 mk bool var 68 arith assert upper 5 datatype splits 6 decisions 14 arith add rows 7 propagations 1 conflicts 10 arith fixed eqs 4 datatype accessor ax 8 arith conflicts 1 datatype constructor ax 16 num allocs 262281512 final checks 6 added eqs 54 del clause 1 arith eq adapter 3 memory 17.740000 max memory 17.740000 Expand start[0.024s]
 not (l1 = [] || l2 = [])
 && Ordinal.count l1 >= 0 && Ordinal.count (List.tl l1) >= 0
 ==> (List.tl l1 = [] || List.tl l2 = [])
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl l1)))
 (Ordinal.Int (Ordinal.count l1)) simplify into ((not
 ((not (l1 = [] || l2 = []) && Ordinal.count l1 >= 0)
 && Ordinal.count (List.tl l1) >= 0)
 || List.tl l1 = [])
 || List.tl l2 = [])
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl l1)))
 (Ordinal.Int (Ordinal.count l1)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`ty_0 list`_1971|
 … expansions unroll expr (|count_`ty_0 list`_1971| (|get.::.1_1942| l1_1960)) expansions unroll expr (|count_`ty_0 list`_1971| l1_1960) expansions unsat (let ((a!1 (= (|count_`ty_0 list`_1971| l1_1960)
 (+ 1 (|count_`ty_0 list`_1971| (|get.…

termination proof Termination proof call `match_cols (List.tl y)` from `match_cols y` original match_cols y sub match_cols (List.tl y) original ordinal Ordinal.Int (Ordinal.count y) sub ordinal Ordinal.Int (Ordinal.count (List.tl y)) path [(List.hd y).0 = None || (List.hd y).1 = None && not (y = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.009s details Expand smt_stats num checks 7 arith assert lower 6 arith pivots 5 rlimit count 5693 mk clause 28 datatype occurs check 66 mk bool var 164 arith assert upper 4 datatype splits 42 decisions 34 arith add rows 6 propagations 51 conflicts 12 arith fixed eqs 3 datatype accessor ax 27 arith conflicts 1 datatype constructor ax 35 num allocs 318142755 final checks 14 added eqs 150 del clause 2 arith eq adapter 3 memory 21.050000 max memory 21.050000 Expand start[0.009s]
 ((List.hd y).0 = None || (List.hd y).1 = None)
 && not (y = []) && Ordinal.count y >= 0 && Ordinal.count (List.tl y) >= 0
 ==> not
 (((List.hd (List.tl y)).0 = None || (List.hd (List.tl y)).1 = None)
 && not (List.tl y = []))
 && not
 (Option.get (List.hd (List.tl y)).0 =
 Option.get (List.hd (List.tl y)).1
 && not
 ((List.hd (List.tl y)).0 = None
 || (List.hd (List.tl y)).1 = None)
 && not (List.tl y = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl y)))
 (Ordinal.Int (Ordinal.count y)) simplify into (not
 (((((List.hd y).0 = None || (List.hd y).1 = None) && not (y = []))
 && Ordinal.count y >= 0)
 && Ordinal.count (List.tl y) >= 0)
 || not
 (((List.hd (List.tl y)).0 = None || (List.hd (List.tl y)).1 = None)
 && not (List.tl y = []))
 && not
 ((Option.get (List.hd (List.tl y)).0 =
 Option.get (List.hd (List.tl y)).1
 && not
 ((List.hd (List.tl y)).0 = None || (List.hd (List.tl y)).1 = None))
 && not (List.tl y = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl y)))
 (Ordinal.Int (Ordinal.count y)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`(ty_0 option * ty_0 option) list`_2002|
 … expansions unroll expr (|count_`(ty_0 option * ty_0 option) list`_2002| (|get.::.1_1984| y_1991)) expansions unroll expr (|count_`(ty_0 option * ty_0 option) list`_2002| y_1991) expansions unsat (let ((a!1 (= (|count_`(ty_0 option * ty_0 option) list`_2002| y_1991)
 (+ 4
 … call `match_cols (List.tl y)` from `match_cols y` original match_cols y sub match_cols (List.tl y) original ordinal Ordinal.Int (Ordinal.count y) sub ordinal Ordinal.Int (Ordinal.count (List.tl y)) path [Option.get (List.hd y).0 = Option.get (List.hd y).1
 && not ((List.hd y).0 = None || (List.hd y).1 = None)
 && not (y = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.021s details Expand smt_stats num checks 7 arith assert lower 5 arith pivots 4 rlimit count 2887 mk clause 26 datatype occurs check 59 mk bool var 147 arith assert upper 5 datatype splits 26 decisions 31 arith add rows 7 propagations 57 conflicts 13 arith fixed eqs 4 datatype accessor ax 26 arith conflicts 1 datatype constructor ax 34 num allocs 298223213 final checks 11 added eqs 148 del clause 2 arith eq adapter 3 memory 18.240000 max memory 20.950000 Expand start[0.021s]
 Option.get (List.hd y).0 = Option.get (List.hd y).1
 && not ((List.hd y).0 = None || (List.hd y).1 = None)
 && not (y = [])
 && Ordinal.count y >= 0 && Ordinal.count (List.tl y) >= 0
 ==> not
 (((List.hd (List.tl y)).0 = None || (List.hd (List.tl y)).1 = None)
 && not (List.tl y = []))
 && not
 (Option.get (List.hd (List.tl y)).0 =
 Option.get (List.hd (List.tl y)).1
 && not
 ((List.hd (List.tl y)).0 = None
 || (List.hd (List.tl y)).1 = None)
 && not (List.tl y = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl y)))
 (Ordinal.Int (Ordinal.count y)) simplify into (not
 ((((Option.get (List.hd y).0 = Option.get (List.hd y).1
 && not ((List.hd y).0 = None || (List.hd y).1 = None))
 && not (y = []))
 && Ordinal.count y >= 0)
 && Ordinal.count (

termination proof Termination proof call `match_rows (List.tl x)` from `match_rows x` original match_rows x sub match_rows (List.tl x) original ordinal Ordinal.Int (Ordinal.count x) sub ordinal Ordinal.Int (Ordinal.count (List.tl x)) path [match_cols (zip (List.hd x).0 (List.hd x).1) && not (x = [])] proof detailed proof summary full ground_instances 5 definitions 0 inductions 0 search_time 0.012s details Expand smt_stats num checks 12 arith assert lower 14 arith pivots 13 rlimit count 6362 mk clause 40 datatype occurs check 72 mk bool var 211 arith assert upper 11 datatype splits 27 decisions 75 arith add rows 29 propagations 34 conflicts 12 arith fixed eqs 6 datatype accessor ax 31 arith conflicts 2 datatype constructor ax 54 num allocs 358250213 final checks 11 added eqs 182 del clause 5 arith eq adapter 8 memory 21.790000 max memory 21.790000 Expand start[0.012s]
 match_cols (zip (List.hd x).0 (List.hd x).1)
 && not (x = []) && Ordinal.count x >= 0 && Ordinal.count (List.tl x) >= 0
 ==> not
 (match_cols (zip (List.hd (List.tl x)).0 (List.hd (List.tl x)).1)
 && not (List.tl x = []))
 || Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl x)))
 (Ordinal.Int (Ordinal.count x)) simplify into (not
 (((match_cols (zip (List.hd x).0 (List.hd x).1) && not (x = []))
 && Ordinal.count x >= 0)
 && Ordinal.count (List.tl x) >= 0)
 || not
 (match_cols (zip (List.hd (List.tl x)).0 (List.hd (List.tl x)).1)
 && not (List.tl x = [])))
|| Ordinal.( << ) (Ordinal.Int (Ordinal.count (List.tl x)))
 (Ordinal.Int (Ordinal.count x)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_102| (|Ordinal.Int_93| (|count_`(ty_0 option list * ty_0 option list) list`_2059|
 … expansions unroll expr (zip_2037 (|tuple_get.0_2020| (|get.::.0_2023| (|get.::.1_2024| x_2050)))
 (|tuple_get.1_20… expansions unroll expr (let ((a!1 (zip_2037 (|tuple_get.0_2020|
 (|get.::.0_2023| (|get.::.1_2024| x_… expansions unroll expr (|count_`(ty_0 option list * ty_0 option list) list`_2059|
 (|get.::.1_2024| x_2050)) expansions unroll expr (|count_`(ty_0 option list * ty_0 option list) list`_2059| x_2050) expansions unsat (let ((a!1 (>= (|count_`ty_0 option list`_2063|
 (|tuple_get.0_2020| (|get.::.0_2023…

## The Satisfaction of Subrepticiously Solving Sudokus using Satisfiability

We can now, finally, ask Imandra to find a sudoku that satisfies all the constraints defined before!

*NOTE* we have to use `[@@bmc]` to use the Bounded Model Checker (BMC), because this problem is prone to combinatorial explosion and is too hard for Imandra's default unrolling algorithm.

In [10]:
instance (fun (s:sudoku) -> is_valid_sudoku s && is_solution_of s the_problem) [@@blast] ;;

- : sudoku -> bool = <fun>
module CX : sig val s : sudoku end


Instance (after 273 steps, 1.522s):
 let s =
 {rows =
 [[Some C8; Some C1; Some C2; Some C7; Some C5; Some C3; Some C6;
 Some C4; Some C9];
 [Some C9; Some C4; Some C3; Some C6; Some C8; Some C2; Some C1;
 Some C7; Some C5];
 [Some C6; Some C7; Some C5; Some C4; Some C9; Some C1; Some C2;
 Some C8; Some C3];
 [Some C1; Some C5; Some C4; Some C2; Some C3; Some C7; Some C8;
 Some C9; Some C6];
 [Some C3; Some C6; Some C9; Some C8; Some C4; Some C5; Some C7;
 Some C2; Some C1];
 [Some C2; Some C8; Some C7; Some C1; Some C6; Some C9; Some C5;
 Some C3; Some C4];
 [Some C5; Some C2; Some C1; Some C9; Some C7; Some C4; Some C3;
 Some C6; Some C8];
 [Some C4; Some C3; Some C8; Some C5; Some C2; Some C6; Some C9;
 Some C1; Some C7];
 [Some C7; Some C9; Some C6; Some C3; Some C1; Some C8; Some C4;
 Some C5; Some C2]]}

Instance

Let us look at the initial sudoku and its solution side to side: 

In [11]:
Imandra.display (Document.tbl_of doc_of_sudoku [[the_problem; CX.s]]) ;;

- : unit = ()


8,·,·,·,·,·,·,·,·
·,·,3,6,·,·,·,·,·
·,7,·,·,9,·,2,·,·
·,5,·,·,·,7,·,·,·
·,·,·,·,4,5,7,·,·
·,·,·,1,·,·,·,3,·
·,·,1,·,·,·,·,6,8
·,·,8,5,·,·,·,1,·
·,9,·,·,·,·,4,·,·
8,1,2,7,5,3,6,4,9
9,4,3,6,8,2,1,7,5


We can manipulate `CX.s` easily, directly in OCaml:

In [12]:
let transpose_sudoku (s:sudoku) : sudoku = {rows = transpose s.rows};;

transpose_sudoku CX.s;;

val transpose_sudoku : sudoku -> sudoku = <fun>
- : sudoku = <document>


8,9,6,1,3,2,5,4,7
1,4,7,5,6,8,2,3,9
2,3,5,4,9,7,1,8,6
7,6,4,2,8,1,9,5,3
5,8,9,3,4,6,7,2,1
3,2,1,7,5,9,4,6,8
6,1,2,8,7,5,3,9,4
4,7,8,9,2,3,6,1,5
9,5,3,6,1,4,8,7,2


In [ ]:
let f x = match x with 
    | None -> true
    | Some y -> 
        begin
            match y with 
            | 0 -> true
            | _ -> false
        end;;

In [ ]:
f 0;;